In [77]:
import pandas as pd
import numpy as np
from glob import glob
from tqdm.notebook import tqdm, trange
import os
import datetime

In [134]:
W, H = 1920, 1080

In [125]:
files = glob('../runs/detect/predict*/labels/*')

In [126]:
len(files)

662844

In [127]:
files = [x for x in files if 'predict5' not in x and 'predict6' not in x]

In [128]:
len(files)

662798

In [129]:
files[:5]

['../runs/detect/predict/labels/cfg_raw_cam_01_fhd_h265_20230609T000003_1.txt',
 '../runs/detect/predict/labels/cfg_raw_cam_01_fhd_h265_20230609T000003_2.txt',
 '../runs/detect/predict/labels/cfg_raw_cam_01_fhd_h265_20230609T000003_3.txt',
 '../runs/detect/predict/labels/cfg_raw_cam_01_fhd_h265_20230609T000003_4.txt',
 '../runs/detect/predict/labels/cfg_raw_cam_01_fhd_h265_20230609T000003_5.txt']

In [137]:
# filename, camera_id, datetime, frame_id, label_class, x,y,x,y, confidance   #;;;;
coco_cols = ['label_class', 'x', 'y', 'w', 'h', 'confidence']
res = []
for x in tqdm(files):
    bname = os.path.basename(x).replace('.txt', '')
    sname = bname.split('_')

    video_file = '_'.join(sname[:-1]) + '.mkv'
    frame = int(sname[-1])
    camera = sname[3]
    dt = sname[6]
    dt_parsed = datetime.datetime.strptime(dt, "%Y%m%dT%H%M%S")
    
    with open(x, 'r') as file:
        lines = [x.rstrip() for x in file.readlines()]
        for l in lines:
            l_split = l.split(' ')
            curr_data = {v: float(l_split[k]) for k, v in enumerate(coco_cols)}
            curr_data['filename'] = video_file
            curr_data['camera_id'] = camera
            curr_data['datetime'] = dt_parsed
            curr_data['frame_id'] = frame
            res.append(curr_data)

df_res = pd.DataFrame.from_records(res).loc[:, ['filename', 'camera_id', 'datetime', 'frame_id', 'label_class', 'x', 'y', 'w', 'h', 'confidence']]
df_res['label_class'] = df_res['label_class'].astype(int)
df_res

  0%|          | 0/662798 [00:00<?, ?it/s]

,filename,camera_id,datetime,frame_id,label_class,x,y,w,h,confidence
0,cfg_raw_cam_01_fhd_h265_20230609T000003.mkv,01,2023-06-09 00:00:03,1,8,0.296998,0.765053,0.201359,0.196389,0.694145
1,cfg_raw_cam_01_fhd_h265_20230609T000003.mkv,01,2023-06-09 00:00:03,2,8,0.296892,0.764497,0.201743,0.197460,0.721214
2,cfg_raw_cam_01_fhd_h265_20230609T000003.mkv,01,2023-06-09 00:00:03,3,8,0.297045,0.765550,0.201709,0.194799,0.691033
3,cfg_raw_cam_01_fhd_h265_20230609T000003.mkv,01,2023-06-09 00:00:03,4,8,0.297082,0.765901,0.201874,0.194279,0.629399
4,cfg_raw_cam_01_fhd_h265_20230609T000003.mkv,01,2023-06-09 00:00:03,5,8,0.297110,0.764586,0.201582,0.196424,0.772130
...,...,...,...,...,...,...,...,...,...,...
826234,cfg_raw_cam_02_fhd_h265_20230707T214000.mkv,02,2023-07-07 21:40:00,2140,8,0.799422,0.761484,0.396978,0.458214,0.324898
826235,cfg_raw_cam_02_fhd_h265_20230707T214000.mkv,02,2023-07-07 21:40:00,2142,8,0.616754,0.554351,0.038852,0.052457,0.353814
826236,cfg_raw_cam_02_fhd_h265_20230707T214000.mkv,02,2023-07-07 21:40:00,2143,8,0.799920,0.761428,0.394584,0.459609,0.263392
826237,cfg_raw_cam_02_fhd_h265_20230707T214000.mkv,02,2023-07-07 21:40:00,2323,8,0.800212,0.761403,0.397113,0.458038,0.326050


In [138]:
df_res['frame_id'] = df_res['frame_id'] - 1

In [139]:
df_res['x'] = (df_res['x'] * W).astype(int)
df_res['y'] = (df_res['y'] * H).astype(int)
df_res['w'] = (df_res['w'] * W).astype(int)
df_res['h'] = (df_res['h'] * H).astype(int)

In [140]:
df_res.head()

,filename,camera_id,datetime,frame_id,label_class,x,y,w,h,confidence
0,cfg_raw_cam_01_fhd_h265_20230609T000003.mkv,01,2023-06-09 00:00:03,0,8,570,826,386,212,0.694145
1,cfg_raw_cam_01_fhd_h265_20230609T000003.mkv,01,2023-06-09 00:00:03,1,8,570,825,387,213,0.721214
2,cfg_raw_cam_01_fhd_h265_20230609T000003.mkv,01,2023-06-09 00:00:03,2,8,570,826,387,210,0.691033
3,cfg_raw_cam_01_fhd_h265_20230609T000003.mkv,01,2023-06-09 00:00:03,3,8,570,827,387,209,0.629399
4,cfg_raw_cam_01_fhd_h265_20230609T000003.mkv,01,2023-06-09 00:00:03,4,8,570,825,387,212,0.772130


In [141]:
df_res.to_csv('../annotations/yolov8x_no_train_labels.csv', sep=';')